<center><img src="https://github.com/MAGALLANESJoseManuel/OpenDataBigData_at_quality4point0/blob/main/logo.png?raw=true
" width="1000"></center>



# The COVID DATA



## ACCESSING COVID DEATHS

The official [site](https://www.datosabiertos.gob.pe/dataset/fallecidos-por-covid-19-ministerio-de-salud-minsa) does not offer a link. The file was save in AWS:

In [ ]:
import time

linkFallecidos="https://short-talks.s3.sa-east-1.amazonaws.com/fallecidos_covid.csv"


import pandas as pd

# ALSO: time it takes
start_time = time.time()
covid_death=pd.read_csv(linkFallecidos)
print(time.time() - start_time, "seconds")

In [ ]:
# check data types
covid_death.info()

As you see, the separator is a semicolon, then:

In [ ]:
covid_death=pd.read_csv(linkFallecidos,sep=';')
covid_death.info()

In [ ]:
# see it:
covid_death.head()

Let's keep complete data only:

In [ ]:
covid_death[covid_death.isnull().any(axis=1)]

In [ ]:
# Then
covid_death=covid_death[~covid_death.isnull().any(axis=1)]

Verify messiness:

In [ ]:
covid_death[covid_death.PROVINCIA.str.contains(r'[^A-Z\sÑ]',regex=True,na=False)]

Get age stats:

In [ ]:
covid_death.EDAD_DECLARADA.describe(include='all')

In [ ]:
covid_death.SEXO.value_counts()

In [ ]:
# sorted(covid_death.PROVINCIA.unique())

In [ ]:
covid_death.CLASIFICACION_DEF.value_counts()

The only obvious formatting is dates:

In [ ]:
dateColumnsDeath=['FECHA_CORTE','FECHA_FALLECIMIENTO']
covid_death.loc[:,dateColumnsDeath].apply(pd.to_datetime)

In [ ]:
# this is the right way:
covid_death.loc[:,dateColumnsDeath].apply(lambda col:pd.to_datetime(col,format='%Y%m%d'))

In [ ]:
#then
covid_death[dateColumnsDeath]=covid_death.loc[:,dateColumnsDeath].apply(lambda col:pd.to_datetime(col,format='%Y%m%d'))
covid_death.info()

The last two columns need not be _float_:

In [ ]:
IntColumnsDeath=['UBIGEO','UUID']
covid_death[IntColumnsDeath]=covid_death.loc[:,IntColumnsDeath].astype(int)
covid_death.info()

In [ ]:
import os

folder = "data"

# Check whether the specified path exists or not
if not os.path.exists(folder):
   # Create a new directory because it does not exist
   os.makedirs(folder)
   covid_death.to_csv(os.path.join('data','covid_death.csv'),index=False)
   covid_death.to_parquet(os.path.join('data','covid_death.parquet'),index=False)

else:
    covid_death.to_csv(os.path.join('data','covid_death.csv'),index=False)
    covid_death.to_parquet(os.path.join('data','covid_death.parquet'),index=False)


## ACCESSING COVID CASES

Let's open the covid cases data (which is much larger):

In [ ]:
linkCases="https://short-talks.s3.sa-east-1.amazonaws.com/positivos_covid.csv"
start_time = time.time()
covid_cases=pd.read_csv(linkCases,sep=';')
print(time.time() - start_time, "seconds")
# check
covid_cases.info()

The large file is not showing counts of missing values, then:

In [ ]:
covid_cases.isnull().sum()

You will not be able to use this at the person level.

In [ ]:
covid_cases.drop(columns=['id_persona'],inplace=True)

Let's see about UBIGEO:

In [ ]:
covid_cases[covid_cases.UBIGEO.isnull()]

If we plan to use provinces, we drop this:

In [ ]:
covid_cases[covid_cases.PROVINCIA=='EN INVESTIGACIÓN']

In [ ]:
# Then
covid_cases.replace('EN INVESTIGACIÓN', None, inplace=True)
covid_cases_Prov=covid_cases[~covid_cases.PROVINCIA.isnull()]

In [ ]:
# verify
covid_cases_Prov.isnull().sum()

Let's check the dates:

In [ ]:
dateColumnsCases=['FECHA_CORTE','FECHA_RESULTADO']
covid_cases_Prov[dateColumnsCases]=covid_cases_Prov.loc[:,dateColumnsCases].apply(lambda col:pd.to_datetime(col,format='%Y%m%d'))
covid_cases_Prov.info()

In [ ]:
# something simple

IntColumnsCases=['EDAD','UBIGEO']
covid_cases_Prov[IntColumnsCases]=covid_cases_Prov.loc[:,IntColumnsCases].astype(int)
covid_cases_Prov.info()

Time to save:

In [ ]:
import os

folder = "data"

# Check whether the specified path exists or not
if not os.path.exists(folder):
   # Create a new directory because it does not exist
   os.makedirs(folder)
   covid_cases_Prov.to_csv(os.path.join('data','covid_cases_Prov.csv'),index=False)
   covid_cases_Prov.to_parquet(os.path.join('data','covid_cases_Prov.parquet'),index=False)

else:
    covid_cases_Prov.to_csv(os.path.join('data','covid_cases_Prov.csv'),index=False)
    covid_cases_Prov.to_parquet(os.path.join('data','covid_cases_Prov.parquet'),index=False)